### Populate file list based on timestamp

In [ ]:
from os import walk
fitsfilesdates = {} 

for (dirpath, dirnames, filenames) in walk('./fitsfiles'):
    for file in filenames:    
        if(file.endswith('.fits')):
            date = file[0:12]
            if(date not in fitsfilesdates):
                fitsfilesdates[date] = []
            if('pfss_intoout' in file):
                fitsfilesdates[date].append((dirpath + '/' + file, 'PFSS_IO'))
            elif('pfss_outtoin' in file):
                fitsfilesdates[date].append((dirpath + '/' + file, 'PFSS_OI'))
            elif('scs_outtoin' in file):
                fitsfilesdates[date].append((dirpath + '/' + file, 'SCS_OI'))
    break
    
for l in fitsfilesdates:
    fitsfilesdates[l].sort()

In [ ]:
velfilesdates = {}
wsafilesdates = {}
for (dirpath, dirnames, filenames) in walk('./fitsfiles_images'):
    for file in filenames:    
        if('vel' in file):
            date = file[4:16]
            velfilesdates[date] = dirpath + '/' + file
        if('wsa' in file):
            date = file[4:16]
            wsafilesdates[date] = dirpath + '/' + file

# Function definitions


In [ ]:
import math, numpy

def sph2cart(coord):
    return [
         coord[2] * math.sin(coord[1]) * math.cos(coord[0]), 
         coord[2] * math.sin(coord[1]) * math.sin(coord[0]),
         coord[2] * math.cos(coord[1])
    ]

def coord2datarow(coord):
    sph = sph2cart(coord[0:3])
    sph.append(coord[5])
    return [numpy.float32(x) for x in sph] 

In [ ]:
from enum import Enum
class Model(Enum):
    Batsrus = 0
    Enlil = 1
    Pfss = 2
    Wsa = 3
    Invalid = 5

In [ ]:
def obstimeToJ2000(ot):
    timefits = ot[0:4] + '-' + ot[5:7] + '-' + ot[8:10] + 'T'
    timefits += ot[11:13] + ot[14:17] + ot[18:21] + '.000'
    pathSafeTimeString = timefits.replace(':', '-')
    time = Time(timefits, format='fits')
    y2000 = Time(2000, format='jyear')
    jdaysdelta = time.jd - y2000.jd
    jsecs = jdaysdelta*60*60*24
    return [jsecs, pathSafeTimeString]

In [ ]:
from astropy.io import fits
from astropy.time import Time
import struct, ctypes, math

def toOsfls(filename, modelname, indices,  typename, xq_filename = ""):
    fl_fits = fits.open(filename)
    fl_data = fl_fits[0].data
    fl_fits.close()
    
    # if no extra arg given, it's the last 540
    if(xq_filename == ""):
        print("sun-earth connection")
    # read file for extra quantity
    elif(modelname == 'PFSS_IO'):
        wsa_fits = fits.open(xq_filename)
        wsa_data = wsa_fits[0].data
        wsa_data = wsa_data[6] # the seventh layer has the open(>=1)/closedness(0)
        wsa_fits.close()
    else:
        vel_fits = fits.open(xq_filename)
        vel_data = vel_fits[0].data
        vel_data = vel_data[1] # the second layer has the wind speed
        vel_fits.close()
        
    
    
    versionNumber = 0
    [triggerTime, pathSafeTimeString] = obstimeToJ2000(fl_fits[0].header['OBSTIME'])
    #print("TriggerTime: ",triggerTime)
    print("pathSafeTimeString: ",pathSafeTimeString)
    fileName = pathSafeTimeString + '.osfls'

    model = Model.Wsa.value
    isMorphable = False

    nVert = 0
    lineStart = []
    lineCount = []
    vertexPositions = []
    extraQuantities = []

    for i in indices:
        points = [coord2datarow(pt) for pt in fl_data[i] if pt[0] > -900] 
        if (len(points) < 2): continue
        lineStart.append(nVert)
        nVert += len(points)
        lineCount.append(len(points))
        [vertexPositions.extend(pt[0:3]) for pt in points] # extend to unfold elements
        if(xq_filename == ""):
            sunearthpoint = -1
            if(i in range(16200, 16380)):
                sunearthpoint = 0
            elif (i in range(16380, 16560)):
                sunearthpoint = 1
            elif (i in range(16560, 16740)):
                sunearthpoint = 2
            xtra = [sunearthpoint]*len(points) # same value for all points
            extraQuantities.extend(xtra)
        elif(modelname == 'PFSS_IO'):
            #[extraQuantities.append(pt[3]) for pt in points]
            openness = wsa_data[math.floor(i/180)][i%180]
            xtra = [openness]*len(points) # same value for all points
            extraQuantities.extend(xtra)
        else:     
            velocity = vel_data[math.floor(i/180)][i%180]
            if(points[0][3] < 0): velocity = -1*velocity
            xtra = [velocity]*len(points) # same value for all points
            extraQuantities.extend(xtra)
    
    nLines = len(lineStart)

    nExtras = 1
    if(xq_filename == ""):
        extraQuantityNames = ['sun-earth level \0']
    elif(modelname == 'PFSS_IO'):
        extraQuantityNames = ['open/closed regions \0']
    else: 
        extraQuantityNames = ['wind speed with polarity \0']
    nStringBytes = sum([len(s) for s in extraQuantityNames])
    allNamesInOne = ''
    for s in extraQuantityNames:
        allNamesInOne += s
    
    # Prepare data for writing to binary. Using Struct and pack
    typestr = '= i d i ? Q Q Q Q %sl %sL %sf %sf %ss' % (nLines, nLines, 3*nVert, nExtras*nVert, nStringBytes)
    struct_to_write = struct.Struct(typestr)
    #print('Format string  :', struct_to_write.format)
    print('Uses           :', struct_to_write.size, 'bytes')
    values_to_write = (versionNumber, triggerTime, model, isMorphable, nLines, nVert, nExtras, nStringBytes)
    values_to_write += (*lineStart, *lineCount, *vertexPositions, *extraQuantities, allNamesInOne.encode('utf-8'))
    
    buffer = ctypes.create_string_buffer(struct_to_write.size)    
    struct_to_write.pack_into(buffer, 0, *values_to_write)
    
    fout = open('./' + modelname + '/' + typename + '_' + fileName, 'wb')
    fout.write(buffer)
    fout.close()


In [ ]:
def everynth(step):
    return (range(0,16200, step), 'step' + str(step))

#### Only PFSS_IO, every nth

In [ ]:
import time
start_time = time.time()
# relies heavily on having a sorted list of files for each timestep
# so that pfss_io goes first and picks out the lines
for timestamp in fitsfilesdates:
    pfss_io = fitsfilesdates[timestamp][0]
    indices = everynth(13) # change step size here
    toOsfls(pfss_io[0], pfss_io[1], indices[0], indices[1], wsafilesdates[timestamp])
    print('Finished converting {} after {} seconds.'.format(pfss_io[1],time.time()-start_time))
    
        
print("Execution time for type {}: {} seconds".format(indices[1], time.time()-start_time))


### Pick closed field lines from PFSS_IO
By removing the open ones. Opens ones are defined by their first and last point having the same polarity.

In [ ]:
def pickClosed(filename):
    fl_fits = fits.open(filename)
    fl_data = fl_fits[0].data

    indices_to_save = []

    for i in range(16200):
        b_radii = [pt[5] for pt in fl_data[i] if pt[0] > -900] 
        if (len(b_radii) < 2): continue
        first_b_radius = b_radii[0]
        last_b_radius = b_radii[-1]
        if(first_b_radius*last_b_radius < 0): #if product is negative/opposite signs
            indices_to_save.append(i)
    return indices_to_save

In [ ]:
# make a pfss_io closed lines sparser
def make_sparser(indices, step):
    new_indices = indices[::3]
    return [new_indices, 'closed_step' + str(step)]

### Pick boundary lines

In [ ]:
import math

def boundaryLines(filename_io, filename_oi, indices_closed):
    fl_io = (fits.open(filename_io))[0].data
    fl_oi = (fits.open(filename_oi))[0].data

    threshold = math.sqrt(2)
    # boundary_lines_io = [] do we want these?
    boundary_lines_oi = []
    io_last_phi = {}
    io_last_theta = {}
    io_first_phi = {}
    io_first_theta = {}

    for j in indices_closed:
        last = -1 # find last index
        for point in fl_io[j]:
            if (point[0] > -900):
                last += 1
            else: break
        # get the first and last coordinates for in-to-out (both on surface)
        io_last_phi[j] = math.degrees(fl_io[j][last][0])
        io_last_theta[j] = math.degrees(fl_io[j][last][1])
        io_first_phi[j] = math.degrees(fl_io[j][0][0])
        io_first_theta[j] = math.degrees(fl_io[j][0][1])

    for i in range(16200):
        last = -1
        for point in fl_oi[i]:
            if (point[0] > -900):
                last += 1
            else: break
        if (last < 2): continue
        # get the last coordinates for out-to-in (the ones on the surface)
        oi_last_phi = math.degrees(fl_oi[i][last][0])
        oi_last_theta = math.degrees(fl_oi[i][last][1])
        for j in indices_closed:
            # calculate distances and compare
            dist_last_phi = abs(io_last_phi[j] - oi_last_phi)
            dist_last_theta = abs(io_last_theta[j] - oi_last_theta)
            dist_first_phi = abs(io_first_phi[j] - oi_last_phi)
            dist_first_theta = abs(io_first_theta[j] - oi_last_theta)
            if(dist_last_phi < threshold and dist_last_theta < threshold):
                #boundary_lines_io.append(j)
                boundary_lines_oi.append(i)
                break
            if(dist_first_phi < threshold and dist_first_theta < threshold):
                #boundary_lines_io.append(j)
                boundary_lines_oi.append(i)
                break
    return [boundary_lines_oi, 'boundary']
    

In [ ]:
def fillOut(boundary_indices, step):
    output = []
    previous_index = 0
    for index in boundary_indices:
        subrange = list(range(previous_index, index, step))
        output.extend(subrange)
        if(output[-1] != index):    
            output.append(index)
        previous_index = index
    return [output, 'boundary_filled']


### Make into osfls - closed lines and boundary lines with fillers

In [ ]:
import time
start_time = time.time()
# relies heavily on having a sorted list of files for each timestep
# so that pfss_io goes first and picks out the lines
for timestamp in fitsfilesdates:
    pfss_io = fitsfilesdates[timestamp][0]
    pfss_oi = fitsfilesdates[timestamp][1]
    scs_oi = fitsfilesdates[timestamp][2]
    indices_closed_lines = pickClosed(pfss_io[0])
    print('Picked closed lines for {} after {} seconds.'.format(timestamp,time.time()-start_time))
    indices = make_sparser(indices_closed_lines, 3) # change here for sparser
    #toOsfls(pfss_io[0], pfss_io[1], indices[0], indices[1], wsafilesdates[timestamp])
    #print('Finished converting {} after {} seconds.'.format(pfss_io[1],time.time()-start_time))
    indices = boundaryLines(pfss_io[0], pfss_oi[0], indices_closed_lines)
    indices = fillOut(indices[0], 25) # change step here for sparser
    print('Picked boundary lines after {} seconds.'.format(time.time()-start_time))
    toOsfls(pfss_oi[0], pfss_oi[1], indices[0], indices[1], velfilesdates[timestamp])
    print('Finished converting {} after {} seconds.'.format(pfss_oi[1],time.time()-start_time))
    toOsfls(scs_oi[0], scs_oi[1], indices[0], indices[1], velfilesdates[timestamp])
    print('Finished converting {} after {} seconds.'.format(scs_oi[1],time.time()-start_time))
        
print("Execution time for type {}: {} seconds".format(indices[1], time.time()-start_time))


### Pick the last 540 (sun-earth connection)
And turn OI sets to osfls

In [ ]:
def mergeAndConvertToOsfls(filenamePFSS, filenameSCS):
    fl_fits_pfss = fits.open(filenamePFSS)
    fl_data_pfss = fl_fits_pfss[0].data
    fl_fits_pfss.close()
    
    fl_fits_scs = fits.open(filenameSCS)
    fl_data_scs = fl_fits_scs[0].data
    fl_fits_scs.close()
    
    indices = range(16200,16740)

    print("sun-earth connection")
        
    
    [triggerTime, pathSafeTimeString] = obstimeToJ2000(fl_fits_pfss[0].header['OBSTIME'])
    [triggerTime_s, pathSafeTimeString_s] = obstimeToJ2000(fl_fits_scs[0].header['OBSTIME'])
    if(triggerTime != triggerTime_s):
        print("Files are from different times. Canceling conversion. triggertimes: ")
        print(triggerTime)
        print(triggerTime_s)
        
    versionNumber = 0
    fileName = pathSafeTimeString + '.osfls'
    model = Model.Wsa.value
    isMorphable = False
    
    nVert = 0
    lineStart = []
    lineCount = []
    vertexPositions = []
    extraQuantities = []
    
    extraQuantities_polarity = []
    extraQuantities_level = []

    for i in indices:
        points_pfss = [coord2datarow(pt) for pt in fl_data_pfss[i] if pt[0] > -900] 
        points_scs = [coord2datarow(pt) for pt in fl_data_scs[i] if pt[0] > -900] 
        
        if (len(points_pfss) < 2 or len(points_scs) < 2): continue
            
        lineStart.append(nVert)
        
        combinedVertLen = (len(points_pfss) + len(points_scs))
        nVert += combinedVertLen
        lineCount.append(combinedVertLen)
        
        [vertexPositions.extend(pt[0:3]) for pt in points_scs] # add scs vertices FIRST
        [vertexPositions.extend(pt[0:3]) for pt in points_pfss] # add pfss vertices
        
        
        # The following methods does not really work if you want to mask away a layer of lines
        #polarity = 0
        #if(points_scs[0][0] >= 0):
        #    polarity = 4
        #sunearthpoint = 3
        #if (i in range(16380, 16560)):
        #    sunearthpoint = 1
        #elif (i in range(16560, 16740)):
        #    sunearthpoint = 3
        
        # middle section of point density is colored by polarity
        #quarter = int(combinedVertLen/4)
        #mid = combinedVertLen - (2*quarter)
        #xtra = [sunearthpoint]*quarter # use sun eart level category for ends of fieldline
        #xtra.extend([polarity]*mid) # use polarity for middle of fieldline
        #xtra.extend([sunearthpoint]*quarter)
        
        # color rings (based on radius) of polarity on line set
        #xtra = []     
        #for point in fl_data_scs[i]:
        #    if(point[0] < -900): continue
        #    if(math.floor(point[2])%2 != 0 or point[2] >19): #if even or above 19 solar radii
        #        xtra.append(sunearthpoint)
        #    else:
        #        xtra.append(polarity)
        #for point in fl_data_pfss[i]:
        #    if(point[0] < -900): continue
        #    if(math.floor(point[2])%2 != 0 ): #if even
        #        xtra.append(sunearthpoint)
        #    else:
        #        xtra.append(polarity)
        
        polarity = 0
        if(points_scs[0][0] >= 0):
            polarity = 1
        xtra = [polarity]*combinedVertLen
        extraQuantities_polarity.extend(xtra)
        
        sunearthpoint = 1
        if (i in range(16380, 16560)):
            sunearthpoint = 0
        elif (i in range(16560, 16740)):
            sunearthpoint = 2
        xtra = [sunearthpoint]*combinedVertLen
        extraQuantities_level.extend(xtra)
       
    nLines = len(lineStart)

    nExtras = 2
    extraQuantities = extraQuantities_polarity
    extraQuantities.extend(extraQuantities_level)

    extraQuantityNames = ['polarity \0', 'sun-earth level\0']

    nStringBytes = sum([len(s) for s in extraQuantityNames])
    allNamesInOne = ''
    for s in extraQuantityNames:
        allNamesInOne += s
    
    # Prepare data for writing to binary. Using Struct and pack
    typestr = '= i d i ? Q Q Q Q %sl %sL %sf %sf %ss' % (nLines, nLines, 3*nVert, nExtras*nVert, nStringBytes)
    struct_to_write = struct.Struct(typestr)
    #print('Format string  :', struct_to_write.format)
    print('Uses           :', struct_to_write.size, 'bytes')
    values_to_write = (versionNumber, triggerTime, model, isMorphable, nLines, nVert, nExtras, nStringBytes)
    values_to_write += (*lineStart, *lineCount, *vertexPositions, *extraQuantities, allNamesInOne.encode('utf-8'))
    
    buffer = ctypes.create_string_buffer(struct_to_write.size)    
    struct_to_write.pack_into(buffer, 0, *values_to_write)
    
    fout = open('./SUN-EARTH/sun-earth-comb_' + fileName, 'wb')
    fout.write(buffer)
    fout.close()

In [ ]:
import time
start_time = time.time()

for timestamp in fitsfilesdates:
    pfss_oi = fitsfilesdates[timestamp][1]
    scs_oi = fitsfilesdates[timestamp][2]
    mergeAndConvertToOsfls(pfss_oi[0], scs_oi[0])       
print("Execution time for type sun earth: {} seconds".format(time.time()-start_time))